In [1]:
import pyspark
import re
import pandas as pd

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, from_json, col
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, IntegerType

In [3]:
spark = SparkSession \
    .builder \
    .appName("Spark_data_clean") \
    .config("spark.master", "spark://rayiMac.modem:7077") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:3.2.2_0.18.0") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.num", "1") \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/ray/.ivy2/cache
The jars for the packages stored in: /Users/ray/.ivy2/jars
com.crealytics#spark-excel_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8e7dd231-4158-428b-b3f3-e0db9594d8c5;1.0
	confs: [default]
	found com.crealytics#spark-excel_2.12;3.2.2_0.18.0 in central
	found org.apache.poi#poi;5.2.2 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.commons#commons-collections4;4.4 in central
	found org.apache.commons#commons-math3;3.6.1 in central
	found commons-io#commons-io;2.11.0 in central
	found com.zaxxer#SparseBitSet;1.2 in central
	found org.apache.logging.log4j#log4j-api;2.17.2 in central
	found org.apache.poi#poi-ooxml;5.2.2 in central
	found org.apache.poi#poi-ooxml-lite;5.2.2 in central
	found org.apache.xmlbeans#xmlbeans;5.0.3 in central
	found org.apache.commons#commons-compress;1.21 in central
	found com.github.virtuald#curvesapi;1.07 in central
	found com.norbi

In [4]:
# load csv file into dataframe
df_metadata = spark.read.csv("movies_metadata.csv", header=True)

In [5]:
# cache dataframe for further processing
df_metadata.cache()

DataFrame[adult: string, belongs_to_collection: string, budget: string, genres: string, homepage: string, id: string, imdb_id: string, original_language: string, original_title: string, overview: string, popularity: string, poster_path: string, production_companies: string, production_countries: string, release_date: string, revenue: string, runtime: string, spoken_languages: string, status: string, tagline: string, title: string, video: string, vote_average: string, vote_count: string]

## Functions 

In [6]:
def transformation(input_df, input_key, col_name, col_alias, input_schema):
    col_name = str(col_name)
    col_alias = str(col_alias)
    primary_key = str(input_key)
    # select col_name in file, import column as json format and apply complex schema to column 
    input_df = input_df.withColumn(col_name, from_json(col(col_name), input_schema))
    # explode the column with primary key, rename the exploded result
    df_exploded = input_df.select(primary_key, explode(col(col_name)).alias(col_alias))
    return df_exploded

## For Collection Column 

In [7]:
# belongs to collection column
collection_schema = ArrayType(
    StructType([
        StructField("id",StringType(), True),
        StructField("name",StringType(), True),
        StructField("poster_path",StringType(), True),
        StructField("backdrop_path",StringType(), True),
    ])
)

In [8]:
# id is the primary key used for this dataframe
df_collection_result = transformation (df_metadata, "id", "belongs_to_collection", "collection", collection_schema)

In [9]:
df_collection_columns= df_collection_result.select("id",col("collection.id").alias("collection_id"), col("collection.name").alias("collection_name"), col("collection.poster_path").alias("collection_poster_path"), col("collection.backdrop_path").alias("collection_backdrop_path"))

In [10]:
df_collection_columns.cache()

DataFrame[id: string, collection_id: string, collection_name: string, collection_poster_path: string, collection_backdrop_path: string]

In [11]:
df_collection_columns.show(truncate=False)

+-----+-------------+--------------------------------+--------------------------------+--------------------------------+
|id   |collection_id|collection_name                 |collection_poster_path          |collection_backdrop_path        |
+-----+-------------+--------------------------------+--------------------------------+--------------------------------+
|862  |10194        |Toy Story Collection            |/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg|/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg|
|15602|119050       |Grumpy Old Men Collection       |/nLvUdqgPgm3F85NMCii9gVFUcet.jpg|/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg|
|11862|96871        |Father of the Bride Collection  |/nts4iOmNnq7GNicycMJ9pSAn204.jpg|/7qwE57OVZmMJChBpLEbJEmzUydk.jpg|
|710  |645          |James Bond Collection           |/HORpg5CSkmeQlAolx3bKMrKgfi.jpg |/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg|
|21032|117693       |Balto Collection                |/w0ZgH6Lgxt2bQYnf1ss74UvYftm.jpg|/9VM5LiJV0bGb1st1KyHA3cVnO2G.jpg|
|9273 |3167         |Ace Ventura

## For Genres Column

In [12]:
df_metadata.select("genres").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------+
|genres                                                                                                                       |
+-----------------------------------------------------------------------------------------------------------------------------+
|[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]                             |
|[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]                            |
|[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]                                                             |
|[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]                                |
|[{'id': 35, 'name': 'Comedy'}]                                                                         

In [13]:
# genres column
genres_schema = ArrayType(
    StructType([
        StructField("id",StringType(), True),
        StructField("name",StringType(), True),
    ])
)

In [18]:
# use transformation function
df_genres_result = transformation(df_metadata, "id", "genres","genres", genres_schema)

In [19]:
df_genres_columns= df_genres_result.select("id",col("genres.id").alias("genres_id"), col("genres.name").alias("genres_name"))

In [20]:
df_genres_columns.show()

+-----+---------+-----------+
|   id|genres_id|genres_name|
+-----+---------+-----------+
|  862|       16|  Animation|
|  862|       35|     Comedy|
|  862|    10751|     Family|
| 8844|       12|  Adventure|
| 8844|       14|    Fantasy|
| 8844|    10751|     Family|
|15602|    10749|    Romance|
|15602|       35|     Comedy|
|31357|       35|     Comedy|
|31357|       18|      Drama|
|31357|    10749|    Romance|
|11862|       35|     Comedy|
|  949|       28|     Action|
|  949|       80|      Crime|
|  949|       18|      Drama|
|  949|       53|   Thriller|
|11860|       35|     Comedy|
|11860|    10749|    Romance|
|45325|       28|     Action|
|45325|       12|  Adventure|
+-----+---------+-----------+
only showing top 20 rows



## production companies

In [21]:
df_metadata.select("production_companies").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|production_companies                                                                                                                                                                                                                                                                                                                 |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[{'name': 'Pixa

In [32]:
# production_companies column
companies_schema = ArrayType(
    StructType([
        StructField("name",StringType(), True),
        StructField("id",StringType(), True),
    ])
)

In [33]:
# use transformation function
df_result = transformation(df_metadata, "id", "production_companies","companies", companies_schema)

In [34]:
df_companies_columns= df_result.select("id",col("companies.id").alias("companies_id"), col("companies.name").alias("companies_name"))

In [35]:
df_companies_columns.show()

+-----+------------+--------------------+
|   id|companies_id|      companies_name|
+-----+------------+--------------------+
|  862|           3|Pixar Animation S...|
| 8844|         559|    TriStar Pictures|
| 8844|        2550|        Teitler Film|
| 8844|       10201|Interscope Commun...|
|15602|        6194|        Warner Bros.|
|15602|       19464|      Lancaster Gate|
|11862|        5842|Sandollar Product...|
|11862|        9195| Touchstone Pictures|
|  949|         508| Regency Enterprises|
|  949|         675|        Forward Pass|
|  949|        6194|        Warner Bros.|
|11860|           4|  Paramount Pictures|
|11860|         258|Scott Rudin Produ...|
|11860|         932|  Mirage Enterprises|
|11860|        5842|Sandollar Product...|
|11860|       14941|Constellation Ent...|
|11860|       55873|           Worldwide|
|11860|       58079|Mont Blanc Entert...|
|45325|           2|Walt Disney Pictures|
| 9091|          33|  Universal Pictures|
+-----+------------+--------------

## production countries

In [54]:
df_metadata.select("production_countries").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|production_countries                                                                                                                                                              |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[{'iso_3166_1': 'US', 'name': 'United States of America'}]                                                                                                                        |
|[{'iso_3166_1': 'US', 'name': 'United States of America'}]                                                                                                                        |
|[{'iso_3166_1': 'US', 'name': 'United States of America'}]                                    

In [61]:
# production_companies column
countries_schema = ArrayType(
    StructType([
        StructField("`iso_3166_1`",StringType(), True),
        StructField("name",StringType(), True),
    ])
)

In [62]:
# use transformation function
df_result = transformation(df_metadata, "id", "production_countries","countires", companies_schema)
df_result.show()

+-----+--------------------+
|   id|           countires|
+-----+--------------------+
|  862|{null, United Sta...|
| 8844|{null, United Sta...|
|15602|{null, United Sta...|
|11862|{null, United Sta...|
|  949|{null, United Sta...|
|11860|     {null, Germany}|
|11860|{null, United Sta...|
|45325|{null, United Sta...|
| 9091|{null, United Sta...|
|  710|{null, United Kin...|
|  710|{null, United Sta...|
| 9087|{null, United Sta...|
|12110|      {null, France}|
|12110|{null, United Sta...|
|21032|{null, United Sta...|
|10858|{null, United Sta...|
| 1408|      {null, France}|
| 1408|     {null, Germany}|
| 1408|       {null, Italy}|
| 1408|{null, United Sta...|
+-----+--------------------+
only showing top 20 rows



In [60]:
df_cuntries_columns= df_result.select("id",col("countires.iso_3166_1"), col("countires.name").alias("countires_name"))

AnalysisException: [FIELD_NOT_FOUND] No such struct field `iso_3166_1` in `iso_31661`, `name`.

In [47]:
df_cuntries_columns.show()

+-----+-------------+--------------------+
|   id|countires_iso|      countires_name|
+-----+-------------+--------------------+
|  862|         null|United States of ...|
| 8844|         null|United States of ...|
|15602|         null|United States of ...|
|11862|         null|United States of ...|
|  949|         null|United States of ...|
|11860|         null|             Germany|
|11860|         null|United States of ...|
|45325|         null|United States of ...|
| 9091|         null|United States of ...|
|  710|         null|      United Kingdom|
|  710|         null|United States of ...|
| 9087|         null|United States of ...|
|12110|         null|              France|
|12110|         null|United States of ...|
|21032|         null|United States of ...|
|10858|         null|United States of ...|
| 1408|         null|              France|
| 1408|         null|             Germany|
| 1408|         null|               Italy|
| 1408|         null|United States of ...|
+-----+----

## Spoken Languages

In [63]:
df_metadata.select("spoken_languages").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------+
|spoken_languages                                                                                                        |
+------------------------------------------------------------------------------------------------------------------------+
|[{'iso_639_1': 'en', 'name': 'English'}]                                                                                |
|[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]                                       |
|[{'iso_639_1': 'en', 'name': 'English'}]                                                                                |
|1995-12-22                                                                                                              |
|[{'iso_639_1': 'en', 'name': 'English'}]                                                                                |
|[{'iso_639_1': 